In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [ ]:
fhm = pd.ExcelFile('https://www.arcgis.com/sharing/rest/content/items/b5e7488e117749c19881cce45db13f7e/data')

fhm.sheet_names

In [ ]:
df =pd.read_excel('https://www.arcgis.com/sharing/rest/content/items/b5e7488e117749c19881cce45db13f7e/data',sheet_name='Antal avlidna per dag',
                 parse_dates=True,skipfooter=1)

In [ ]:
df


In [ ]:
df.set_index('Datum_avliden',inplace=True)

In [ ]:
df

In [ ]:
df.plot(figsize=(18,12))
plt.grid(True)

In [ ]:
df_weekly = df.resample('W').sum()

In [ ]:
df_weekly

In [ ]:
title = 'SWE Weekly Covid deaths YTD {}'.format(df_weekly.index[-2].date())
df_weekly[:-1].plot(style='o--',figsize=(18,12),title=title)
plt.grid(True)

In [ ]:
conf_df = pd.read_excel('https://www.arcgis.com/sharing/rest/content/items/b5e7488e117749c19881cce45db13f7e/data',sheet_name='Antal per dag region',
                       usecols=[0,1],index_col=0,parse_dates=True)

In [ ]:
conf_df

In [ ]:
conf_weekly=conf_df.resample('W').sum()
conf_weekly

In [ ]:

title = 'SWE Weekly Covid positive YTD {}'.format(conf_weekly.index[-2].date())
conf_weekly[:-1].plot(style='o--',figsize=(18,12),title=title)
plt.grid(True)

In [ ]:
icu = fhm.parse('Antal intensivvårdade per dag', parse_dates=True,
               index_col=0)

icu

In [ ]:
icu.plot(figsize=(18,12))

In [ ]:
icu_weekly = icu.resample('W').sum()
icu_weekly

In [ ]:
title = 'SWE Weekly Covid ICU YTD {}'.format(icu_weekly.index[-2].date())

icu_weekly[:-1].plot(figsize=(18,12),style='o--',title=title)

In [ ]:
scb = pd.ExcelFile('https://scb.se/hitta-statistik/statistik-efter-amne/befolkning/befolkningens-sammansattning/befolkningsstatistik/pong/tabell-och-diagram/preliminar-statistik-over-doda/')

In [ ]:
scb.sheet_names

In [ ]:
scb_daily = scb.parse('Tabell 1',skiprows=6)
scb_daily.index = pd.date_range('2020-01-01' , '2021-01-01')
scb_daily = scb_daily.drop('DagMånad',axis=1)
scb_daily = scb_daily.loc[:,'2015' : '2015-2019']
unknowns = scb_daily.iloc[-1]
scb_daily = scb_daily.iloc[:-1]
scb_daily

In [ ]:
knowns_sum = scb_daily.sum()
knowns_sum

In [ ]:
daily_pct = scb_daily / knowns_sum
daily_pct

In [ ]:
scb_daily_corr = scb_daily + unknowns * daily_pct
scb_daily_corr

In [ ]:
corr_diff = scb_daily_corr - scb_daily
corr_diff

In [ ]:
corr_diff.plot(figsize=(18,12))

In [ ]:
scb_daily_corr.sum()

In [ ]:
baseline = scb_daily['2015-2019']
baseline

In [ ]:
pct_change = scb_daily_corr.div(baseline,axis=0)
pct_change

In [ ]:
pct_change.plot(y='2020',figsize=(18,12))
plt.axhline(1,ls='dashed',color='k')

In [ ]:
weekly_change = pct_change.resample('W').mean()
weekly_change

In [ ]:
last_data_week = weekly_change.index[-1]
last_good_week = last_data_week - pd.DateOffset(weeks=2)
last_good_week

In [ ]:
weekly_change[:last_good_week].plot(figsize=(18,12),y='2020',style='o--')
plt.axhline(1,ls='dashed',color='k')

In [ ]:
cum_growth = scb_daily_corr.cumsum().div(baseline.cumsum(),axis=0)
cum_growth

In [ ]:
cum_growth.plot(y='2020', figsize=(18,12))
plt.axhline(1,ls='dashed',color='k')

In [ ]:
weekly_dead = scb_daily.resample('W').sum()
weekly_dead

In [ ]:
weekly_cum_growth = weekly_dead['2020'].cumsum().div(
    weekly_dead['2015-2019'].cumsum())

weekly_cum_growth

In [ ]:
weekly_cum_growth[:last_good_week].plot(figsize=(18,12),style='o--')
plt.axhline(1,ls='dashed',color='k')

In [ ]:
monthly_dead = scb_daily.groupby(scb_daily.index.month).sum()
monthly_dead

In [ ]:
monthly_cum_growth = monthly_dead['2020'].cumsum().div(monthly_dead['2015-2019'].cumsum())
monthly_cum_growth

In [ ]:
monthly_cum_growth.plot(figsize=(18,12),style='o--')
plt.axhline(1,ls='dashed',color='k')

In [ ]:
weekly_fhm = pd.concat([conf_weekly,icu_weekly,df_weekly],axis=1)
weekly_fhm.columns = ['Covid_Pos','Covid_ICU','Covid_Dead']
weekly_fhm

In [ ]:
import requests
pop_url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/FolkmangdNov'
pop_json_query = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:RegionRiket99",
        "values": [
          "00"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:ÅlderTotA",
        "values": [
          "tot"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2020"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(pop_url, json = pop_json_query)
r.status_code

In [ ]:
pop_json = r.json()
pop_json['data'][0]

In [ ]:
pop_df = pd.DataFrame.from_dict(pop_json['data'])
pop_df

In [ ]:
key = pd.DataFrame(pop_df['key'].to_list(),columns=['dummy','dummy2','gender','year'])
key.drop(['dummy','dummy2'],axis=1,inplace=True)
key

In [ ]:
pop_df['pop'] = pop_df['values'].apply(lambda x : x[0])
pop_df['pop'] = pop_df['pop'].astype(int)
pop_df.drop(['key','values'],inplace=True,axis=1)
pop_df

In [ ]:
pop_df = pd.concat([key,pop_df],axis=1).groupby('year').sum()
pop_df

In [ ]:
weekly_fhm


In [ ]:
weekly_fhm_cumsum = weekly_fhm.cumsum()
weekly_fhm_cumsum

In [ ]:
pop_df = pd.Series(pop_df['pop'])


In [ ]:
pop_df

In [ ]:
fhm_pct_impacted = weekly_fhm_cumsum / pop_df.values[0]
fhm_pct_impacted

In [ ]:
plt.figure(figsize=(18,12))

plt.title('SWEDEN')
plt.plot(fhm_pct_impacted['Covid_Pos'],weekly_fhm['Covid_Pos'] / (pop_df.values[0] / 1e6),'o--')
plt.ylabel('weekly increment Covid Positive per Million')
plt.xlabel('proportion of population Covid Positive')
plt.savefig('swe_covid_inflection_point.jpg',format='jpg')

In [ ]:
df


In [ ]:

ax = df.plot(figsize=(18,12))


In [ ]:
plt.figure(figsize=(18,12))
plt.plot(df['Antal_avlidna'].values / df['Antal_avlidna'].max())
freq = 1/30
x = np.linspace(0,len(df),1000)
y = np.cos(freq * x)
#plt.plot(x,y)

In [ ]:
from scipy.fft import rfft,rfftfreq,irfft

sample_rate = 30 # 30 daily data points for a month

signal = df['Antal_avlidna']
signal = signal - signal.mean()

signal = signal.values

yf = rfft(signal) 

xf = rfftfreq(len(signal),1 / sample_rate)

In [ ]:
plt.figure(figsize=(18,12))
plt.plot(xf,np.abs(yf),'o--')

In [ ]:
30 /0.10452962 # a tenth of the period done in a month gives a period of 287 days

In [ ]:
pd.date_range('2020-03-11',periods=287,freq='D')

In [ ]:
max_power_idx = yf.argmax()

new_signal = np.zeros_like(yf)
new_signal[max_power_idx] = yf[max_power_idx]



In [ ]:
plt.plot(xf,np.abs(new_signal))

In [ ]:
yf_filtered = irfft(new_signal)

plt.figure(figsize=(18,12))
plt.plot(yf_filtered / yf_filtered.max() + 1) 
plt.plot(df['Antal_avlidna'].values / df['Antal_avlidna'].values.max() + 1)


